In [1]:
from nltk.tokenize import word_tokenize

from embedding import E5LargeEmbeddingFunction

import clickhouse_connect
import pandas as pd
import logging

client = clickhouse_connect.get_client(host='y1jzidyt9q.us-east-2.aws.clickhouse.cloud', port=8443, username='default', password='_lQ_JWXYQD3ym')

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
data = pd.read_csv('Corpus_v4.csv')
data.shape

(32103, 4)

In [18]:
data.dropna(subset=['text'], inplace=True)
data.drop_duplicates(subset='text', inplace=True)
data = data[data['text'].str.strip() != '']
data.reset_index(drop=True, inplace=True)
data.shape

(29242, 4)

In [4]:
def get_nltk_chunks(text, chunk_size=200, chunk_overlap=100, language="russian"):
    words = word_tokenize(text, language=language)

    chunks = []

    end_index = chunk_size
    while end_index - chunk_size < len(words):
        sentence = ''
        tokens_counter = 0
        for word in words[end_index - chunk_size:end_index]:
            if word in ',.?!:)»' or (sentence and sentence[-1] in '«('):
                sentence += word
            else:
                sentence += " " + word
            tokens_counter += 1

        if tokens_counter < chunk_size and chunks:
            words = word_tokenize(chunks[-1] + sentence)[-chunk_size:]
            sentence = ''
            for word in words:
                if word in ',.?!:)»' or (sentence and sentence[-1] in '«('):
                    sentence += word
                else:
                    sentence += " " + word
                    
        chunks.append(sentence)
        end_index += chunk_size - chunk_overlap

    return chunks

In [5]:
logging.basicConfig(level=logging.INFO, filename='db.log', filemode="a",
                       format="%(asctime)s %(levelname)s %(message)s")

emb_func = E5LargeEmbeddingFunction()

emb_func.change_mode(new_mode='passage')

In [14]:
for index, row in data[:3].iterrows(): 
    try:
        if index % 25 == 0:
            logging.info(f'{index} done')
        link = row.link
        text = row.text
        pages = row.pages

        if not isinstance(pages, int):
            pages = 0

        is_markdown = row.is_markdown

        if is_markdown:
            chunks = get_nltk_chunks(text, chunk_size = 2000, chunk_overlap = 100)
        else:
            chunks = get_nltk_chunks(text, chunk_size = 250, chunk_overlap = 15)

        insert_data = []
        for i, chunk in enumerate(chunks):
            insert_data.append((index, i, link, chunk, pages, emb_func(chunk)))
    
        client.insert('index_texts', insert_data, column_names=['ID', 'chunk_id', 'link', 'text', 'pages', 'embedding'])
    except Exception as e:
        logging.error(index)

asdass
asdass
asdass


In [ ]:
client.insert('index_texts', data, column_names=['ID', 'chunk_id', 'link', 'text','embedding'])

In [ ]:
result = client.query('''SELECT
    *,
    L2Distance(embedding, [0.1, 0.2, 0.3]) AS score
FROM index_texts
ORDER BY score ASC
LIMIT 5''')
result.result_rows

[(1,
  1,
  'link',
  'лол хочу кредит',
  0,
  [3.6540000438690186, 2.453000068664551, 3.2309999465942383],
  5.1281269785525865)]